**INSTALL PYSPARK**

In [3]:
! pip install pyspark

**START SPARK SESSION**

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("StockDataProject") \
    .getOrCreate()

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkException: Invalid Spark URL: spark://HeartbeatReceiver@Wesley_ruan:58222
	at org.apache.spark.rpc.RpcEndpointAddress$.apply(RpcEndpointAddress.scala:66)
	at org.apache.spark.rpc.netty.NettyRpcEnv.asyncSetupEndpointRefByURI(NettyRpcEnv.scala:140)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.executor.Executor.<init>(Executor.scala:301)
	at org.apache.spark.scheduler.local.LocalEndpoint.<init>(LocalSchedulerBackend.scala:64)
	at org.apache.spark.scheduler.local.LocalSchedulerBackend.start(LocalSchedulerBackend.scala:132)
	at org.apache.spark.scheduler.TaskSchedulerImpl.start(TaskSchedulerImpl.scala:235)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:599)
	at org.apa

**API REQUEST - BATCH PROCESSING**

In [45]:
import requests
import json

API_KEY = "42JIJGRH51L2GBTP"
SYMBOL = "IBM"
INTERVAL = "30min"
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={SYMBOL}&interval={INTERVAL}&apikey={API_KEY}'

response = requests.get(url)
data = response.json()

time_series_key = f"Time Series ({INTERVAL})"
if time_series_key not in data:
    print("Error: Invalid response or requests limit exceeded.")
else:
    print("Success!")

records = []
for timestamp, values in data[time_series_key].items():
    records.append((
        SYMBOL,
        timestamp,
        float(values["1. open"]),
        float(values["2. high"]),
        float(values["3. low"]),
        float(values["4. close"]),
        int(values["5. volume"])
    ))

columns = ["symbol", "timestamp", "open", "high", "low", "close", "volume"]
df = spark.createDataFrame(records, schema=columns)

df.show()

Tabela criada com sucesso!


**SEND DATA TO KAFKA**

In [46]:
from confluent_kafka import Producer
import json

producer_config = {
    'bootstrap.servers': 'localhost:9092',
    'client.id': 'api-producer'
}
producer = Producer(producer_config)

topic_name = 'stock_data_topic'

for row in df.collect():
    record = {
        "symbol": row["symbol"],
        "timestamp": row["timestamp"],
        "open": row["open"],
        "high": row["high"],
        "low": row["low"],
        "close": row["close"],
        "volume": row["volume"]
    }
    producer.produce(topic_name, value=json.dumps(record))

producer.flush()
print("Data sent to Kafka!")

SUcess!


symbol,timestamp,open,high,low,close,volume
str,str,f64,f64,f64,f64,i64
"""IBM""","""2025-01-28 19:30""",225.37,225.55,225.35,225.5,120
"""IBM""","""2025-01-28 19:00""",225.66,225.66,225.32,225.5,881926
"""IBM""","""2025-01-28 18:30""",225.66,225.66,225.316,225.55,881838
"""IBM""","""2025-01-28 18:00""",225.48,225.49,225.4,225.4,381
"""IBM""","""2025-01-28 17:30""",225.43,225.55,225.31,225.48,205


**CONSUME DATA FROM KAKFA USING PYSPARK**

In [47]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType, IntegerType

schema = StructType([
    StructField("symbol", StringType()),
    StructField("timestamp", TimestampType()),
    StructField("open", DoubleType()),
    StructField("high", DoubleType()),
    StructField("low", DoubleType()),
    StructField("close", DoubleType()),
    StructField("volume", IntegerType())
])

kafka_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "stock_data_topic") \
    .load() \
    .selectExpr("CAST(value AS STRING)") \
    .select(from_json("value", schema).alias("data")) \
    .select("data.*")

def write_to_postgres(df, epoch_id):
    df.write \
        .format("jdbc") \
        .option("url", "jdbc:postgresql://localhost:5432/mydatabase") \
        .option("dbtable", "stock_data") \
        .option("user", "root") \
        .option("password", "root") \
        .mode("append") \
        .save()

query = kafka_df.writeStream \
    .foreachBatch(write_to_postgres) \
    .start()

query.awaitTermination()

Send to Kafka!


**QUERY DATA**

In [48]:
df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/mydatabase") \
    .option("dbtable", "stock_data") \
    .option("user", "root") \
    .option("password", "root") \
    .load()

df.show(10)

Inserido: 2025-01-28 19:30
Inserido: 2025-01-28 19:00
Inserido: 2025-01-28 18:30
Inserido: 2025-01-28 18:00
Inserido: 2025-01-28 17:30
Inserido: 2025-01-28 17:00
Inserido: 2025-01-28 16:30
Inserido: 2025-01-28 16:00
Inserido: 2025-01-28 15:30
Inserido: 2025-01-28 15:00
Inserido: 2025-01-28 14:30
Inserido: 2025-01-28 14:00
Inserido: 2025-01-28 13:30
Inserido: 2025-01-28 13:00
Inserido: 2025-01-28 12:30
Inserido: 2025-01-28 12:00
Inserido: 2025-01-28 11:30
Inserido: 2025-01-28 11:00
Inserido: 2025-01-28 10:30
Inserido: 2025-01-28 10:00
Inserido: 2025-01-28 09:30
Inserido: 2025-01-28 09:00
Inserido: 2025-01-28 08:30
Inserido: 2025-01-28 08:00
Inserido: 2025-01-28 07:30
Inserido: 2025-01-28 07:00
Inserido: 2025-01-28 06:30
Inserido: 2025-01-28 06:00
Inserido: 2025-01-28 05:30
Inserido: 2025-01-28 05:00
Inserido: 2025-01-28 04:30
Inserido: 2025-01-28 04:00
Inserido: 2025-01-27 19:30
Inserido: 2025-01-27 19:00
Inserido: 2025-01-27 18:30
Inserido: 2025-01-27 18:00
Inserido: 2025-01-27 17:30
I

**FINISH SPARK SESSION**

In [55]:
spark.stop()

shape: (5, 8)
┌─────┬────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────────┬────────┐
│ id  ┆ symbol ┆ timestamp    ┆ open         ┆ high         ┆ low          ┆ close        ┆ volume │
│ --- ┆ ---    ┆ ---          ┆ ---          ┆ ---          ┆ ---          ┆ ---          ┆ ---    │
│ i64 ┆ str    ┆ datetime[μs] ┆ decimal[*,4] ┆ decimal[*,4] ┆ decimal[*,4] ┆ decimal[*,4] ┆ i64    │
╞═════╪════════╪══════════════╪══════════════╪══════════════╪══════════════╪══════════════╪════════╡
│ 800 ┆ IBM    ┆ 2025-01-23   ┆ 225.7500     ┆ 225.9800     ┆ 225.7500     ┆ 225.9800     ┆ 53     │
│     ┆        ┆ 18:00:00     ┆              ┆              ┆              ┆              ┆        │
│ 799 ┆ IBM    ┆ 2025-01-23   ┆ 226.0400     ┆ 226.0400     ┆ 225.7700     ┆ 225.7700     ┆ 690546 │
│     ┆        ┆ 18:30:00     ┆              ┆              ┆              ┆              ┆        │
│ 798 ┆ IBM    ┆ 2025-01-23   ┆ 226.0400     ┆ 226.0400     ┆ 225.8300     ┆ 

C:\Users\ruanw\AppData\Local\Temp\ipykernel_1420\1105828744.py:14: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  df = pl.DataFrame(rows, schema=columns)
